In [1]:
# Lab 9 XOR
import tensorflow as tf
import numpy as np
import pandas as pd
tf.set_random_seed(777)  # for reproducibility

def min_max_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)


xy1=pd.read_csv('datasets_Wine.csv')
xy=xy1.values
for i in range(0,xy.shape[0]):
    if(xy[i][-1]=='good'):
        xy[i][-1]=1
    elif(xy[i][-1]=='bad'):
        xy[i][-1]=0
xy[:,:-1]=min_max_scaler(xy[:,:-1])
test_line=int(len(xy)*0.7)
x_data=xy[:test_line,:-1]
y_data=xy[:test_line,[-1]] 
x_test=xy[test_line:,:-1]
y_test=xy[test_line:,[-1]]
x_data = np.array(x_data, dtype=np.float32) 
y_data = np.array(y_data, dtype=np.float32) 
x_test = np.array(x_test, dtype=np.float32) 
y_test = np.array(y_test, dtype=np.float32) 
X = tf.placeholder(tf.float32, [None,x_data.shape[1]])
Y = tf.placeholder(tf.float32, [None, 1])

C:\Users\USER\Anaconda3\envs\su\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\USER\Anaconda3\envs\su\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\USER\Anaconda3\envs\su\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\USER\Anaconda3\envs\su\lib\site-packages\tensorflow\python\framework\dtypes.

In [2]:
keep_prob = tf.placeholder(tf.float32)
W1 = tf.get_variable("W1", shape=[x_data.shape[1], 100],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([100]))
L1 = tf.sigmoid(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[100,50],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([50]))
L2 = tf.sigmoid(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[50,25],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([25]))
L3 = tf.sigmoid(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[25,10],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([10]))
L4 = tf.sigmoid(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[10, 1],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(L4, W5) + b5
save_file = './train_model_Wine_dropout.ckpt'
saver = tf.train.Saver()

# cost/loss function
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(5001):
        _, cost_val = sess.run([train, cost], feed_dict={X: x_data, Y: y_data,keep_prob:0.7})
        if step % 100 == 0:
            print(step, cost_val)

    # Accuracy report
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_test, Y: y_test,keep_prob:1}
    )
    
    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")
    saver.save(sess, save_file)

0 0.7852982
100 0.70489275
200 0.69838613
300 0.6954486
400 0.6953344
500 0.6953581
600 0.6831794
700 0.6194902
800 0.56877863
900 0.5577188
1000 0.55295014
1100 0.55956453
1200 0.55328465
1300 0.54265517
1400 0.53648454
1500 0.54242027
1600 0.53660804
1700 0.5352086
1800 0.533715
1900 0.5279314
2000 0.54143625
2100 0.5482939
2200 0.520966
2300 0.5315789
2400 0.5375077
2500 0.5325496
2600 0.53639966
2700 0.5172209
2800 0.52797455
2900 0.529128
3000 0.532124
3100 0.5358592
3200 0.53879964
3300 0.5248149
3400 0.5155528
3500 0.5287921
3600 0.5209974
3700 0.5478724
3800 0.5187097
3900 0.528836
4000 0.52404565
4100 0.51876503
4200 0.52411985
4300 0.5237874
4400 0.5256538
4500 0.5166913
4600 0.52339876
4700 0.52395386
4800 0.52022547
4900 0.52161515
5000 0.5150092

Hypothesis:
[[ 2.7936065 ]
 [ 3.0136623 ]
 [ 2.895475  ]
 [ 2.9356406 ]
 [ 2.362885  ]
 [ 1.3678886 ]
 [ 2.7763748 ]
 [ 3.0352485 ]
 [ 1.0620843 ]
 [ 0.16178155]
 [ 0.60503274]
 [ 0.5269111 ]
 [-0.9519459 ]
 [ 3.0512683 ]
 [ 2.234